In [277]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Softmax

from itertools import product

In [278]:
hi = 20
prods = list(product(*[range(1, hi) for _ in range(3)]))

true = [p for p in prods if (sum(p) <= hi and sum(p) > 1 and np.all([k == p[0] for k in p]))]
false = [p for p in prods if (sum(p) <= hi and sum(p) > 1 and np.any([k != p[0] for k in p]))]

true_labels = np.ones(len(true))
false_labels = np.ones(len(false))

# x = np.concatenate()

hi = 100
prods = list(product(*[range(1, hi) for _ in range(3)]))

true_test = [p for p in prods if (sum(p) <= hi and sum(p) > 20 and np.all([k == p[0] for k in p]))]
false_test = [p for p in prods if (sum(p) <= hi and sum(p) > 20 and np.any([k != p[0] for k in p]))]

In [279]:
def md(a,b, c):
    return np.concatenate((np.tile([0,1],a), np.tile([1,0],b), np.tile([1,0],c), np.tile([0,0],20-a-b - c))).reshape(-1,2)

def mt(a,b,c):
    return np.concatenate((np.tile([0,1],a), np.tile([1,0],b), np.tile([1,0],c), np.tile([0,0],100-a-b-c))).reshape(-1,2)


In [280]:

X_true = np.array([md(*t) for t in true])
XXX = np.array([md(*f) for f in false])
X_false = XXX[np.random.choice([i for i in range(len(X_true))], len(X_true))]

y_true = np.ones(len(X_true))
y_false = np.zeros(len(X_false))



X_true_test = np.array([mt(*t) for t in true_test])
XXX_test = np.array([mt(*f) for f in false_test])
X_false_test = XXX_test[np.random.choice([i for i in range(len(X_true_test)*100)], len(X_true_test)*100)]

y_true_test = np.ones(len(X_true_test))
y_false_test = np.zeros(len(X_false_test))



In [281]:
shuffle = np.random.permutation(len(X_true) + len(X_false))


X_train = np.concatenate((X_true, X_false))[shuffle]
y_train = np.concatenate((y_true, y_false))[shuffle]

In [282]:
models =[]
for i in range(3):
    regressor = Sequential()

    regressor.add(LSTM(units = 50, return_sequences = True, input_shape = (20, 2)))
    regressor.add(Dropout(0.2))

    regressor.add(Dense(units = 1))

    regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

    regressor.fit(X_train, y_train, epochs = 200, batch_size = 32, verbose = False)
    
    models.append(regressor)

In [283]:
print(regressor(X_true_test).shape, regressor(X_false_test).shape)

In [284]:
sum(fsize<4)

In [288]:
all_f1 = []
for regressor in models:
    f1s = []
    
    regt = regressor(X_true_test)[:,-1,:] > 0.5
    regf = regressor(X_false_test)[:,-1,:] < 0.5

    fsize = np.array([np.sum(x!=[0,0]) for x in X_false_test])
    tsize = np.array([np.sum(x!=[0,0]) for x in X_true_test])

    for i in range(21,70):
        if i%3 != 0:
            continue
        labels = np.concatenate((y_true_test[tsize == i], y_false_test[fsize == i]))
        guesses = np.concatenate((regt[tsize == i],regf[fsize == i]))
        f1s.append( f1_score(labels, guesses) )
    
    all_f1.append(np.array(f1s))


In [291]:
plt.errorbar(range(20,70,3),np.mean(all_f1,axis = 0), yerr = np.std(all_f1,axis = 0), fmt= ".")
plt.xlabel("Word length")
plt.ylabel("F1")
plt.title("LSTM F1-Score as a function of word-length")